# Introduction

Do higher film budgets lead to more box office revenue? Let's find out if there's a relationship using the movie budgets and financial performance data that I've scraped from [the-numbers.com](https://www.the-numbers.com/movie/budgets) on **May 1st, 2018**. 

<img src=https://i.imgur.com/kq7hrEh.png>

# Import Statements

In [1]:
import pandas as pd
import matplotlib.pyplot as plt


# Notebook Presentation

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Read the Data

In [47]:
data = pd.read_csv('cost_revenue_dirty.csv')
data.head()
data.info()
data.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   5391 non-null   int64 
 1   Release_Date           5391 non-null   object
 2   Movie_Title            5391 non-null   object
 3   USD_Production_Budget  5391 non-null   object
 4   USD_Worldwide_Gross    5391 non-null   object
 5   USD_Domestic_Gross     5391 non-null   object
dtypes: int64(1), object(5)
memory usage: 252.8+ KB


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
4349,568,3/7/2013,Despicable Me 2,"$76,000,000","$975,216,835","$368,065,385"


# Explore and Clean the Data

**Challenge**: Answer these questions about the dataset:
1. How many rows and columns does the dataset contain?
2. Are there any NaN values present?
3. Are there any duplicate rows?
4. What are the data types of the columns?

In [56]:
total_rows = data.shape[0]
total_cols = data.shape[1]
print(f'Total rows: {total_rows}')
print(f'Total columns: {total_cols}')



has_nan_values = data.isna().values.any()
has_any_duplicates = data.duplicated().values.any()
print(f"Total Duplicates: {data.duplicated().sum()}")

print(f'Has NaN values: {has_nan_values}')
print(f"Data Types: {data.dtypes}")
print(f'Has any duplicates: {has_any_duplicates}')

data.describe()

Total rows: 5391
Total columns: 6
Total Duplicates: 0
Has NaN values: False
Data Types: Rank                      int64
Release_Date             object
Movie_Title              object
USD_Production_Budget    object
USD_Worldwide_Gross      object
USD_Domestic_Gross       object
dtype: object
Has any duplicates: False


,Rank
count,"5,391.00"
mean,"2,696.00"
std,"1,556.39"
min,1.00
25%,"1,348.50"
50%,"2,696.00"
75%,"4,043.50"
max,"5,391.00"


### Data Type Conversions

**Challenge**: Convert the `USD_Production_Budget`, `USD_Worldwide_Gross`, and `USD_Domestic_Gross` columns to a numeric format by removing `$` signs and `,`. 
<br>
<br>
Note that *domestic* in this context refers to the United States.

In [57]:
data["USD_Production_Budget"] = data["USD_Production_Budget"].str.replace(',', '').str.replace('$', '').astype('float')
data["USD_Worldwide_Gross"] = data["USD_Worldwide_Gross"].str.replace(',', '').str.replace('$', '').astype('float')
data["USD_Domestic_Gross"] = data["USD_Domestic_Gross"].str.replace(',', '').str.replace('$', '').astype('float')

**Challenge**: Convert the `Release_Date` column to a Pandas Datetime type. 

In [21]:
print(f"Data Types: {data.dtypes}")

Data Types: Rank                       int64
Release_Date              object
Movie_Title               object
USD_Production_Budget    float64
USD_Worldwide_Gross      float64
USD_Domestic_Gross       float64
dtype: object


In [58]:
data["Release_Date"] = pd.to_datetime(data["Release_Date"])

### Descriptive Statistics

**Challenge**: 

1. What is the average production budget of the films in the data set?
2. What is the average worldwide gross revenue of films?
3. What were the minimums for worldwide and domestic revenue?
4. Are the bottom 25% of films actually profitable or do they lose money?
5. What are the highest production budget and highest worldwide gross revenue of any film?
6. How much revenue did the lowest and highest budget films make?

In [ ]:
avg_budget = data["USD_Production_Budget"].mean()
avg_worldwide_gross = data["USD_Worldwide_Gross"].mean()
min_domestic_revenue = data["USD_Domestic_Gross"].sum()
min_worldwide_revenue = data["USD_Worldwide_Gross"].sum()

data.describe(include = 'all')
print(data["Movie_Title"].value_counts())

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
count,"5,391.00",5391,5391,"5,391.00","5,391.00","5,391.00"
unique,NaN,NaN,5318,NaN,NaN,NaN
top,NaN,NaN,Ben-Hur,NaN,NaN,NaN
freq,NaN,NaN,3,NaN,NaN,NaN
mean,"2,696.00",2003-09-19 15:02:02.203672704,NaN,"31,113,737.58","88,855,421.96","41,235,519.44"
min,1.00,1915-08-02 00:00:00,NaN,"1,100.00",0.00,0.00
25%,"1,348.50",1999-12-02 12:00:00,NaN,"5,000,000.00","3,865,206.00","1,330,901.50"
50%,"2,696.00",2006-06-23 00:00:00,NaN,"17,000,000.00","27,450,453.00","17,192,205.00"
75%,"4,043.50",2011-11-23 00:00:00,NaN,"40,000,000.00","96,454,455.00","52,343,687.00"
max,"5,391.00",2020-12-31 00:00:00,NaN,"425,000,000.00","2,783,918,982.00","936,662,225.00"


In [45]:
bottom25_threshold = data["USD_Production_Budget"].quantile(0.25)
bottom25 = data[data["USD_Production_Budget"] <= bottom25_threshold]
print(bottom25)

profitable = [series for series in bottom25 if series['USD_Production_Budget'] > (series['USD_Worldwide_Gross'] + series['USD_Domestic_Gross'])]

#print(f"Profit/Loss: {bottom25['USD_Production_Budget'].sum() - (bottom25['USD_Worldwide_Gross'].sum()+data['USD_Domestic_Gross'].sum())}")
print(profitable)

      Rank Release_Date                     Movie_Title  \
0     5293   1915-08-02           The Birth of a Nation   
1     5140   1916-05-09                     Intolerance   
2     5230   1916-12-24    20,000 Leagues Under the Sea   
3     5299   1920-09-17  Over the Hill to the Poorhouse   
4     5222   1925-01-01                  The Big Parade   
...    ...          ...                             ...   
5332  4020   2017-03-17            The Belko Experiment   
5360  4832   2017-07-03                       Departure   
5371  4901   2017-07-28          An Inconvenient Sequel   
5374  4237   2017-08-25            Polina danser sa vie   
5376  4004   2017-09-06               It Comes at Night   

      USD_Production_Budget  USD_Worldwide_Gross  USD_Domestic_Gross  
0                110,000.00        11,000,000.00       10,000,000.00  
1                385,907.00                 0.00                0.00  
2                200,000.00         8,000,000.00        8,000,000.00  
3      

TypeError: string indices must be integers

# Investigating the Zero Revenue Films

**Challenge** How many films grossed $0 domestically (i.e., in the United States)? What were the highest budget films that grossed nothing?

**Challenge**: How many films grossed $0 worldwide? What are the highest budget films that had no revenue internationally?

### Filtering on Multiple Conditions

**Challenge**: Use the [`.query()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) to accomplish the same thing. Create a subset for international releases that had some worldwide gross revenue, but made zero revenue in the United States. 

Hint: This time you'll have to use the `and` keyword.

### Unreleased Films

**Challenge**:
* Identify which films were not released yet as of the time of data collection (May 1st, 2018).
* How many films are included in the dataset that have not yet had a chance to be screened in the box office? 
* Create another DataFrame called data_clean that does not include these films. 

In [ ]:
# Date of Data Collection
scrape_date = pd.Timestamp('2018-5-1')

### Films that Lost Money

**Challenge**: 
What is the percentage of films where the production costs exceeded the worldwide gross revenue? 

# Seaborn for Data Viz: Bubble Charts

### Plotting Movie Releases over Time

**Challenge**: Try to create the following Bubble Chart:

<img src=https://i.imgur.com/8fUn9T6.png>



# Converting Years to Decades Trick

**Challenge**: Create a column in `data_clean` that has the decade of the release. 

<img src=https://i.imgur.com/0VEfagw.png width=650> 

Here's how: 
1. Create a [`DatetimeIndex` object](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) from the Release_Date column. 
2. Grab all the years from the `DatetimeIndex` object using the `.year` property.
<img src=https://i.imgur.com/5m06Ach.png width=650>
3. Use floor division `//` to convert the year data to the decades of the films.
4. Add the decades as a `Decade` column to the `data_clean` DataFrame.

### Separate the "old" (before 1969) and "New" (1970s onwards) Films

**Challenge**: Create two new DataFrames: `old_films` and `new_films`
* `old_films` should include all the films before 1969 (up to and including 1969)
* `new_films` should include all the films from 1970 onwards
* How many films were released prior to 1970?
* What was the most expensive film made prior to 1970?

# Seaborn Regression Plots

**Challenge**: Use Seaborn's `.regplot()` to show the scatter plot and linear regression line against the `new_films`. 
<br>
<br>
Style the chart

* Put the chart on a `'darkgrid'`.
* Set limits on the axes so that they don't show negative values.
* Label the axes on the plot "Revenue in \$ billions" and "Budget in \$ millions".
* Provide HEX colour codes for the plot and the regression line. Make the dots dark blue (#2f4b7c) and the line orange (#ff7c43).

Interpret the chart

* Do our data points for the new films align better or worse with the linear regression than for our older films?
* Roughly how much would a film with a budget of $150 million make according to the regression line?

# Run Your Own Regression with scikit-learn

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**: Run a linear regression for the `old_films`. Calculate the intercept, slope and r-squared. How much of the variance in movie revenue does the linear model explain in this case?

# Use Your Model to Make a Prediction

We just estimated the slope and intercept! Remember that our Linear Model has the following form:

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**:  How much global revenue does our model estimate for a film with a budget of $350 million? 